# load libraries and data directory

In [1]:
suppressMessages(library(oligo)) 
suppressMessages(library(RankProd)) 
suppressMessages(library(limma))
##suppressMessages(library(biomaRt))

library(gplots)
library(plotrix)
library(RColorBrewer)
library(methods)
library(edgeR)
library(ggbiplot)
library(genefilter)

suppressMessages(library(gtools)) 
suppressMessages(library(Hmisc))
suppressMessages(library(minet))
suppressMessages(library(reshape2))
suppressMessages(library(plyr))
suppressMessages(library(outliers))
suppressMessages(library(magrittr))
suppressMessages(library(dplyr))
suppressMessages(library(tidyr))
suppressMessages(library(purrr))
suppressMessages(library(stringr))

library(tibble)
library(readr)

options(stringsAsFactors = FALSE)


Attaching package: ‘gplots’

The following object is masked from ‘package:IRanges’:

    space

The following object is masked from ‘package:stats’:

    lowess


Attaching package: ‘plotrix’

The following object is masked from ‘package:gplots’:

    plotCI

Warning message:
: package ‘RColorBrewer’ was built under R version 3.3.0Loading required package: ggplot2
Loading required package: plyr

Attaching package: ‘plyr’

The following object is masked from ‘package:oligo’:

    summarize

The following object is masked from ‘package:XVector’:

    compact

The following object is masked from ‘package:IRanges’:

    desc

The following object is masked from ‘package:S4Vectors’:

    rename

Loading required package: scales

Attaching package: ‘scales’

The following object is masked from ‘package:plotrix’:

    rescale

Loading required package: grid

Attaching package: ‘genefilter’

The following object is masked from ‘package:base’:

    anyNA

Warning message:
: package ‘lattice’ w

In [2]:
mRNA_data_dir = '/home/guillaume/data/GSK_enfisema/arrays_mRNA/'
mRNA_u219_annotation_dir = '/home/guillaume/data/GSK_enfisema/'
pheno_csv = '/home/guillaume/data/GSK_enfisema/pheno_groups.csv'
working_dir = '/home/guillaume/Documents/GSK_enfisema/'

# read U219 array cell files and RMA normalize

## create phenotypic data_frame

In [3]:
groups = read.csv(pheno_csv,header = TRUE)

fi = list.files(mRNA_data_dir)[grepl('.CEL',list.files(mRNA_data_dir))]

pheno = data_frame(sample = fi) %>% rowwise %>% 
  mutate(patient = strsplit(sample,split='_')[[1]][1],      
         pat = gsub('RF-','',patient),
         pat = gsub('RF','',pat),
         pat = gsub('-1','',pat),
         pat = gsub('-2','',pat),
         pat = gsub('-3','',pat)) %>% ungroup %>% select(-patient)

#pheno$pat[pheno$pat %in% groups$pat]
#pheno$pat[!pheno$pat %in% groups$pat]
#groups$pat[!groups$pat %in% pheno$pat]

pheno %<>% left_join(groups, by='pat') 
write_csv(pheno, str_c(working_dir,'pheno.csv'))

In [4]:
pheno %>% dim

[1] 94 18

## read CEL files

In [5]:
#U219 Affymetrix array
ph <- new("AnnotatedDataFrame", data=pheno)
affy = read.celfiles(str_c(mRNA_data_dir,ph$sample), 
                             phenoData=ph, sampleNames=ph$pat, verbose=F)
affy_rma = rma(affy)

Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/101_G03.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/102_G04.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/103_G05.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/104_G06.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/106_T-32_(106)_G08.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/107_G09.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/108_T-42_(108)_G10.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/109_T-31_(109)_G11.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/10_A09.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/110_T-30_(110)_G12.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/111_H01.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/112_H02.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/113_H03.CEL
Reading in : /home/guillaume/data/

Warning message:
In read.celfiles(str_c(mRNA_data_dir, ph$sample), phenoData = ph, : 'channel' automatically added to varMetadata in phenoData.

Background correcting
Normalizing
Calculating Expression


In [6]:
affy_rma %>% head

ExpressionSet (storageMode: lockedEnvironment)
assayData: 1 features, 94 samples 
  element names: exprs 
protocolData
  rowNames: 1 2 ... 94 (94 total)
  varLabels: exprs dates
  varMetadata: labelDescription channel
phenoData
  rowNames: 1 2 ... 3 (94 total)
  varLabels: sample pat ... batch (18 total)
  varMetadata: labelDescription channel
featureData: none
experimentData: use 'experimentData(object)'
Annotation: pd.hg.u219 

In [7]:
data_rma = as.data.frame(exprs(affy_rma)[,])
pheno = pData(phenoData(affy_rma))
pheno %<>% mutate(DLCO = ifelse(is.na(DLCO),00,DLCO), ID = str_c(pat,DLCO,sep='_'))
colnames(data_rma) = pheno$ID
data_rma[1:5,1:5]

,101_41,102_59,103_42,104_48,106_50
11715100_at,4.161239,3.760154,4.413014,4.461558,4.109672
11715101_s_at,5.063633,4.853304,4.980275,5.116125,5.075902
11715102_x_at,3.660032,3.905466,3.782236,3.969975,3.682786
11715103_x_at,4.036723,4.062408,3.963214,4.181952,4.226437
11715104_s_at,4.150746,3.876433,3.959996,4.268308,4.005087


# mapping probe id to gene id with affymetrix annotation csv

## load annotation file

In [30]:
probe_entrez = read_csv(
str_c(mRNA_u219_annotation_dir,'HG-U219.na36.annot_affymetrixwebsite_march2016.csv'),
                       skip=25)
probe_entrez = probe_entrez[,c(1,15,19)] %>% as_data_frame
colnames(probe_entrez) = c('probe_set_id','gene_symbol','entrez')
probe_entrez = data_frame(probe_set_id = row.names(data_rma)) %>% left_join(probe_entrez)
write_csv(probe_entrez, str_c(working_dir,'probe_entrez.csv'))

sum(row.names(data_rma) == probe_entrez$probe_set_id)
sum(!row.names(data_rma) == probe_entrez$probe_set_id)
probe_entrez %>% head

Joining, by = "probe_set_id"


[1] 49386

[1] 0

,probe_set_id,gene_symbol,entrez
1,11715100_at,HIST1H3G,8355
2,11715101_s_at,HIST1H3G,8355
3,11715102_x_at,HIST1H3G,8355
4,11715103_x_at,TNFAIP8L1,126282
5,11715104_s_at,OTOP2,92736
6,11715105_at,C17orf78,284099


## filter probes: 
- remove probes without gene symbol or without entrez id
- keep only one probe per gene symbol/entrez id

In [57]:
probe_entrez %>% dim
probe_entrez_f = probe_entrez %<>% rowwise %>% mutate(entrez = as.character(entrez),
                entrez = str_replace_all(entrez,' ',''),
                gene_symbol = str_replace_all(gene_symbol,' ',''),
                entrez = word(entrez,1,sep = '///'),data_rma_f
                gene_symbol = word(gene_symbol,1,sep = '///')) %>%
ungroup %>%
filter(entrez != '---', gene_symbol != '---', !is.na(entrez),
       !is.na(gene_symbol),
      entrez != '', gene_symbol != '')  %>% distinct(entrez, .keep_all = TRUE) %>% 
distinct(gene_symbol, .keep_all = TRUE)

write_csv(probe_entrez_f, str_c(working_dir,'probe_entrez_filtered.csv'))
probe_entrez_f %>% dim

[1] 48873     3

[1] 19289     3

In [67]:
data_rma_f = data_rma[row.names(data_rma) %in% probe_entrez_f$probe_set_id,]
probe_entrez_f = data_frame(probe_set_id = row.names(data_rma_f)) %>% left_join(probe_entrez_f)

sum(row.names(data_rma_f) == probe_entrez_f$probe_set_id)
sum(!row.names(data_rma_f) == probe_entrez_f$probe_set_id)
row.names(data_rma_f) = probe_entrez_f$gene_symbol
data_rma_f[1:5,1:5]

Joining, by = "probe_set_id"


[1] 19289

[1] 0

,101_41,102_59,103_42,104_48,106_50
HIST1H3G,4.161239,3.760154,4.413014,4.461558,4.109672
TNFAIP8L1,4.036723,4.062408,3.963214,4.181952,4.226437
OTOP2,4.150746,3.876433,3.959996,4.268308,4.005087
C17orf78,3.923032,3.918216,4.194412,4.028379,4.023533
CTAGE15,4.268796,4.570817,4.292928,4.827885,4.984516


### filter patients to match miRNA data